                                _____________________priyanthan____________________

______________________________________________________________________________________________________________________________________________________

# Leveraging Multi-Agent AI Systems for Blog Writing

____________________________________________________________________________________________________________________________________________________

### Building 4 seperates agents 
1. resercher
2. planner
3. writer
4. editor

### Defining seperate Tools for Agents
1. ScrapeWebsiteTool()
2. DirectoryReadTool()
3. SerperDevTool() - used the serper API
4. CSVSearchTool()

### Defining each seperate Tasks
1. research_task
2. planning_task
3. writing_task
4. editing_task

### Writing a blog regarding analysis of world happiness report

1. Introduction - what is the world happiness report. and how it is calculated. what are the factors helps to determine the world happiness of each countries. 
2. should compare the previous world happiness reports with 2024 world happiness report. should provide a analysis. 
3. should talk about a particular country's position in 2024 and previous world happiness reports. should provide a clear analysis regarding the it's position. for this it may use the values in the csv file and the pdf file of 2024 report. 
4. should talk about the how to increase that country's position in coming years and what are the changes need to be done.
5. finally it write the conclusion



### Inputs
1. csv file
2. website url
3. country

In [104]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [105]:
from crewai import Agent, Task, Crew

### Model and API keys

In [121]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = '<Your API key>'

### Tools

In [122]:
from crewai_tools import (CSVSearchTool, DirectoryReadTool, 
                            FileReadTool,   ScrapeWebsiteTool, SerperDevTool)

In [123]:
# this is the link of the website that containes all the world happiness reports
scrape_tool = ScrapeWebsiteTool(
    website_url="https://worldhappiness.report/archive/"
)
directory_read_tool = DirectoryReadTool(directory='')
search_tool = SerperDevTool() 
csv_search_tool=CSVSearchTool() #encoding= "ISO-8859-1"

### Defining the Agent

In [124]:
## defining the researcher agent
researcher = Agent(
    role="researcher of world happiness index every year",
    goal="Make sure to do amazing analysis on "
         "the world happiness index to write a blog",
    verbose=True,
    tools = [search_tool, directory_read_tool, 
             scrape_tool, csv_search_tool],
    backstory=(
        "As a researcher of world happiness index "
        "your process in navigating and extracting critical "
        "information from websites, files, reports, {csv_file} "
        "most parcularly you should sear all the important details of "
        "{country} country to specify in the blog. "
    )
)

In [125]:
## defining the planner agent 
planner = Agent(
    role="content planner",
    goal="Plan engaging and factually accurate content on world happiness index",
    verbose=True,
    tools = [search_tool,scrape_tool],
    backstory=(
        "You're working on planning a blog article "
        "about the world happiness index"
        "You collect information that helps the "
        "audience learn about the world happiness index"
        "and make informed decisions. "
        "Your work is the basis for "
        "the Content Writer to write an article on this topic."
        "your should plan to analys the {country}'s happiness index in 2024 and in the past."
        "it should be a details analysis."
        ),
    allow_delegation=False
)

In [126]:
## defining the writer agent
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: world happiness index",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: world happiness index. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [127]:
## defining the proof-reader agent
editor = Agent(
    role="blog editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

### Defining the task

In [128]:
## defining the reserch task
research_task = Task(
    description=(
        "Analyze the URL provided (https://worldhappiness.report/archive/) "
        "to extract the details of world happiness index of countries, "
        "main factors which will dominates to calculate the index. "
        "use this {csv_file} to get world happiness index of 2024. this is the latest list."
        "use the 2024 world happiness report also "
        "use this sources and tools to do the analysis"
        "Also mainly focus on {country}'s index and how it's index in the previous years detaily"
        " Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of  required countries and it's indexes and the factors which used to "
        "calculate the world happiness index. and detailed information about the {country}"
    ),
    agent=researcher
)

In [129]:
## defining the planning task
planning_task = Task(
    description =(
        "1. write the introduction using what is the world happiness report. and how it is calculated." 
            "what are the factors helps to determine the world happiness of each countries.\n" 
        "2. should compare the previous world happiness reports with 2024 world happiness report. " 
            "should provide a analysis.\n"
        "3. should talk about a {country}s position in 2024 and previous world happiness reports." 
            "should provide a clear analysis regarding the it's position. for this it may use the values in the "
            " {csv_file} and the world happiness 2024 report.\n" 
        "4. should talk about the how to increase that {country}'s position in coming years "
            " and what are the changes need to be done.\n"
        "5. finally it write the conclusion\n"
        "6. Identify the target audience, considering "
            "their interests and pain points.\n"
        "7. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [130]:
## defining the writing task
writing_task = Task(
     description=(
        "1. Use the content plan to craft a compelling "
            "blog post on world happiness indexes of countries .\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs."
        "write very detailly in the professional manner",
    human_input=True,
    agent=writer,
)

In [131]:
## defining the proof-reading task
editing_task = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    human_input=True,
    output_file="blog.md",
    agent=editor
)

### Crew

In [132]:
crew = Crew(
    agents=[researcher, planner, writer,editor],
    tasks=[research_task, planning_task, writing_task, editing_task],
    verbose=2,
)

2024-05-23 08:03:21,836 - 14232 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


#### Analysis of Sri Lanka

In [139]:
inputs = {
    "csv_file": "World-happiness-report-2024.csv",
    "country" : "Sri Lanka",
}

In [140]:
output = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: researcher of world happiness index every year
 [INFO]: == Starting Task: Analyze the URL provided (https://worldhappiness.report/archive/) to extract the details of world happiness index of countries, main factors which will dominates to calculate the index. use this World-happiness-report-2024.csv to get world happiness index of 2024. this is the latest list.use the 2024 world happiness report also use this sources and tools to do the analysisAlso mainly focus on Sri Lanka's index and how it's index in the previous years detaily Use the tools to gather content and identify and categorize the requirements.


> Entering new CrewAgentExecutor chain...
I need to gather information on world happiness indexes of different countries, focusing on Sri Lanka in particular, and the main factors used to calculate the index. I should use the World-happiness-report-2024.csv to get the latest data for 2024.

Action: Read website content
Action Input: {"url": "https://wor

In [141]:
from IPython.display import Markdown
Markdown(output)

## The Significance of World Happiness Indexes 

### Introduction

The World Happiness Report is a pivotal publication that assesses the happiness levels of individuals in various countries, providing crucial insights into global well-being. By ranking countries based on their citizens' self-perceived happiness, the report illuminates the factors that influence happiness. Drawing on global survey data, the report considers elements like social support, benevolence, and community well-being to determine a country's happiness score.

### Comparison of World Happiness Reports

Released annually, the World Happiness Report delves into the evolving dynamics of happiness on a global scale, exploring themes such as the repercussions of COVID-19, migration patterns, and community well-being. These reports offer a comprehensive overview of happiness levels across nations, providing valuable insights into the factors that impact individual happiness. Through the analysis of these reports, policymakers and researchers can gain a deeper understanding of societal well-being and pinpoint areas for enhancement.

### Sri Lanka's Position in the World Happiness Report

Although specific data on Sri Lanka's placement in the 2024 report is currently unavailable, previous reports have indicated fluctuations in the country's happiness index. Elements like social norms, conflicts, governmental policies, and technological advancements have influenced Sri Lanka's ranking in the past. It is essential to examine Sri Lanka's standing in the most recent report to assess the country's advancements in promoting well-being and to identify areas for further development.

### Improving Sri Lanka's Happiness Index

In order to elevate Sri Lanka's standing in the World Happiness Report, policymakers should prioritize initiatives that foster social support, community well-being, and benevolence. The implementation of policies focusing on mental health, education, and social equity can significantly impact the country's happiness index. Encouraging activities that cultivate a sense of belonging and interpersonal connection among individuals can also contribute to a higher happiness ranking for Sri Lanka.

### Conclusion

In conclusion, the World Happiness Report serves as a valuable instrument for evaluating national well-being and grasping the factors that influence happiness levels. A thorough analysis of Sri Lanka's position in the report can offer valuable insights into the country's progress and areas requiring attention. By emphasizing social well-being, mental health, and community support, Sri Lanka can strive towards enhancing its happiness index and fostering a happier society.

### Target Audience

This article is tailored to policymakers, researchers, and individuals with an interest in global well-being. It caters to those seeking to comprehend the factors that shape happiness levels in different countries and those looking for insights on augmenting societal well-being.

By incorporating SEO keywords such as World Happiness Report, happiness index, global well-being, Sri Lanka's happiness ranking, and factors influencing happiness, this blog post aims to reach a broader audience intrigued by the subject. Proofread for grammatical errors and ensure alignment with the brand's voice before publication.

#### Analysis of America

In [136]:
inputs = {
    "csv_file": "World-happiness-report-2024.csv",
    "country" : "america"
}

In [137]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: researcher of world happiness index every year
 [INFO]: == Starting Task: Analyze the URL provided (https://worldhappiness.report/archive/) to extract the details of world happiness index of countries, main factors which will dominates to calculate the index. use this World-happiness-report-2024.csv to get world happiness index of 2024. this is the latest list.use the 2024 world happiness report also use this sources and tools to do the analysisAlso mainly focus on america's index and how it's index in the previous years detaily Use the tools to gather content and identify and categorize the requirements.


> Entering new CrewAgentExecutor chain...
I need to gather information on the world happiness index of countries, the main factors used to calculate the index, and specifically focus on America's index and its trends over the years. I will use the tools available to extract this information from the World Happiness Report website and the provided CSV file

In [138]:
from IPython.display import Markdown
Markdown(result)

# The World Happiness Index: A Deep Dive into Global Happiness Levels

## Introduction

The World Happiness Report is a significant tool that provides valuable insights into the happiness levels of countries worldwide. Utilizing factors such as social support, GDP per capita, healthy life expectancy, freedom to make life choices, generosity, and perceptions of corruption, the report calculates the world happiness index. This index sheds light on the universal desire for happiness and the ability of individuals to support each other in times of need.

## Analysis of the 2024 World Happiness Report

Comparing the 2024 World Happiness Report with previous editions reveals a shift towards analyzing the happiness of individuals across various age groups. The report emphasizes the importance of well-being initiatives and the role of social support in promoting happiness. It underscores the interconnectedness of factors influencing happiness and highlights the need for a holistic approach to well-being.

In the 2024 World Happiness Report, countries are ranked based on their happiness index score, with Finland consistently holding the top spot in recent years. The report also highlights the impact of income disparities, social support systems, and overall life satisfaction on a country's happiness ranking.

## America's Position in the World Happiness Index

America's position in the 2024 World Happiness Report is analyzed in-depth, considering its ranking in previous years. The report delves into the factors contributing to America's overall happiness score, providing insights into areas of strength and opportunities for improvement. The United States, while ranking relatively high in GDP per capita, faces challenges related to income inequality and social support systems that impact its happiness ranking.

## Strategies to Increase America's Happiness Ranking

To elevate America's position in the World Happiness Index, it is crucial to focus on enhancing social support, addressing income disparities, and fostering a sense of community. By prioritizing these factors and implementing targeted strategies, America can work towards a higher happiness ranking in the future. Well-being initiatives play a crucial role in promoting happiness and improving overall quality of life for individuals across the country.

## Conclusion

In conclusion, the World Happiness Report serves as a valuable resource for policymakers, researchers, and individuals interested in well-being and happiness metrics. By analyzing America's position in the 2024 report and comparing it with previous years, valuable insights can be gained to drive initiatives that enhance happiness levels in the country. With a focus on social support, well-being initiatives, and community engagement, America can strive towards a higher happiness ranking in the global index.

The World Happiness Index provides a comprehensive overview of happiness levels worldwide, guiding efforts to promote well-being and happiness for all individuals. By understanding the factors influencing happiness and implementing targeted strategies, countries can work towards a happier and more fulfilling future for their citizens.